
---
# Load needed libraries and functions

In [1]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
%pprint

notebook_path = get_notebook_path()
print(notebook_path)

s = Storage()
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])
hunting_df = s.load_object('hunting_df')
basic_quals_dict = s.load_object('basic_quals_dict')
dir()

Pretty printing has been turned OFF
D:\Documents\Repositories\notebooks\Miscellaneous\ipynb\Job Hunting.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']


['Config', 'CountVectorizer', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', 'TfidfTransformer', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_ih', '_ii', '_iii', '_oh', 'basic_quals_dict', 'check_4_doubles', 'check_for_typos', 'conjunctify_list', 'copyfile', 'csv', 'encoding', 'exit', 'get_classifier', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_module_version', 'get_notebook_path', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'hunting_df', 'ipykernel', 'json', 'jupyter_config_dir', 'notebook_path', 'notebookapp', 'np', 'os', 'pd', 'pickle', 'preprocess_data', 'print_all_files_ending_starting_with', 'print_all_files_ending_with', 'print_all_files_starting_with', 'quit', 're', 'remove_empty_folders', 's', 'similar', '

In [69]:

!start %windir%\explorer.exe "{os.path.abspath(os.path.dirname(notebook_path))}"


---
# Run these to clear out the dataset for the demo

In [58]:

columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                'Job Posting Title', 'Safi Recommendation', 'Recruiting Start Date', 'Account Group', 'Job Requisition ID',
                'Job Type', 'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                'Resource Manager', 'Primary Location', 'Job Description', 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                'Required Clearance', 'Primary Recruiter', 'percent_fit', 'is_opportunity_application_emailed', 'opportunity_application_email_date',
                'is_remote_delivery', 'manager_notes', 'CS Notes']
hunting_df = pd.DataFrame([], columns=columns_list)

In [65]:

[qual_str for qual_str in basic_quals_dict if 'python' in qual_str.lower()][0]

'3+ years of experience with developing software in object-oriented and scripting languages, including MATLAB, C/C++, and Python'

In [53]:

old_key = '3 years of experience with developing software in object-oriented and scripting languages, including MATLAB, C/C++, and Python'
if old_key in basic_quals_dict:
    basic_quals_dict.pop(old_key)
req_id_list = ['R0069317']
mask_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[mask_series, 'percent_fit'] = np.nan


---
# Needed extra functions

In [3]:

# Email prep
subject_str = '{}% fit: Internal Candidate, Dave Babbitt, for {}'
concerns_str = 'One important question I have is if the work can be supported remotely or if this position is available for remote delivery '
concerns_str += '(or something equivalent).'
concerns_str += " I don't want to move my family out of New England."
text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
emails_dir = os.path.join(s.saves_folder, 'emails')
os.makedirs(name=emails_dir, exist_ok=True)
name_regex = re.compile(r'^([^(]+) \((\d+)\)')
def clean_email(email_str):
    match_obj = name_regex.search(email_str)
    if match_obj:
        email_str = match_obj.group(1).strip()
        employee_id = match_obj.group(2).strip()
        names_list = re.split(r'\s+', email_str, 0)
        if len(names_list) >= 2:
            first_name = names_list[0]
            last_name = names_list[1]
            email_str = '{}, {} [USA] <{}@bah.com>'.format(last_name, first_name, employee_id)
    
    return email_str

In [4]:

def get_percent_fit(row_series):
    percent_fit = row_series['percent_fit']
    if str(percent_fit) == 'nan':
        percent_fit = 0
    percent_fit = int(percent_fit*100)
    
    return percent_fit

In [5]:

def print_loc_computation(row_index, quals_list, basic_quals_dict):
    print()
    numerator_str_list = []
    for qual_str in quals_list:
        if qual_str in basic_quals_dict:
            numerator_str_list.append(str(basic_quals_dict[qual_str]))
        else:
            numerator_str_list.append('000')
    numerator_str = '+'.join(numerator_str_list)
    print("hunting_df.loc[{}, 'percent_fit'] = ({})/{}".format(row_index, numerator_str, len(quals_list)))

In [6]:

def get_predictions_and_counts(prediction_list, quals_list):
    qual_count = 0
    prediction_str = ''
    for pred_array, qual_str in zip(prediction_list, quals_list):
        prediction = pred_array[1]
        prediction_str += '\n{} {}'.format(prediction, qual_str)
        if prediction > 0.5:
            qual_count += 1
    
    return prediction_str, qual_count

In [7]:

def get_quals_str(prediction_list, quals_list, basic_quals_dict):
    qual_count = 0
    quals_str = ''
    for pred_array, (i, qual_str) in zip(prediction_list, enumerate(quals_list)):
        if qual_str in basic_quals_dict:
            formatted_str = '\nquals_list[{}] = "{}" ({})'
        else:
            formatted_str = '\n*quals_list[{}] = "{}" ({})'
        prediction = pred_array[1]
        quals_str += formatted_str.format(i, qual_str, prediction)
        if prediction > 0.5:
            qual_count += 1
    
    return quals_str, qual_count

In [8]:

def print_fit_job(row_index, row_series, basic_quals_dict):
    job_fitness = 0.0
    job_description = row_series['Job Description']
    quals_list = get_quals_list(job_description)
    if len(quals_list):
        prediction_list = list(predict_percent_fit(quals_list))
        #prediction_str, qual_count = get_predictions_and_counts(prediction_list, quals_list)
        quals_str, qual_count = get_quals_str(prediction_list, quals_list, basic_quals_dict)
        job_fitness = qual_count/len(prediction_list)
        if job_fitness > 0.8:
            print('Basic Qualifications:{}'.format(quals_str))
            #print(prediction_str)
            print(job_fitness)
            print_loc_computation(row_index, quals_list, basic_quals_dict)
    
    return quals_list, job_fitness

In [9]:

def qual_sum(qual_str):
    results = '"{}"'.format(qual_str)
    if qual_str in basic_quals_dict:
        results = basic_quals_dict[qual_str]
    else:
        results = predict_percent_fit([qual_str])[0][1]
        if results > 0.5:
            results = 1.0
        else:
            results = 0.0
    
    return str(results)

In [10]:

def print_job_description(req_id):
    mask_series = (hunting_df['Job Requisition ID'] == req_id)
    job_description = hunting_df[mask_series]['Job Description'].tolist()[0]
    print(get_quals_list(job_description))
    print(job_description)

In [11]:

scanner_regex = re.compile(r'\b[1-9a-zA-Z][0-9a-zA-Z]*( *[#\+]{1,2}|\b)')
def regex_tokenizer(corpus):
    
    return [match.group() for match in re.finditer(scanner_regex, corpus)]

In [12]:

import string

printable_regex = re.compile('[^{}]+'.format(string.printable))
def un_msword_ify(x):
    msword_str = str(x)
    msword_str = printable_regex.sub(r' ', msword_str).strip()
    msword_str = re.sub(r'[^\x00-\x7f]+', r' ', msword_str).strip()
    msword_str = re.sub(r' +', ' ', msword_str)
    msword_str = re.sub(r'::', ':', msword_str)
    msword_str = re.sub(r':$', '', msword_str)
    msword_str = re.sub(r'^-', '', msword_str)
    
    return msword_str

In [13]:

a_list = ['Additional Qualifications?', 'Nice If You Have', 'Nice if you have', 'Nice if You Have',
          'Additional Preferred Qualifications', 'Nice if you Have', 'Additional qualifications', 'Nice to Have']
a_str = '({}):?'.format('|'.join(a_list))
def get_quals_list(job_description):
    job_description = un_msword_ify(job_description)
    basic_quals = ''
    quals_list = []
    items_list = re.split('(Key Role|The Challenge):', job_description, 0)
    if len(items_list) > 1:
        job_description = items_list[-1].strip()
    items_list = re.split('[\r\n]+(Basic Qualifications?|You Have|You have):?', job_description, 0)
    if len(items_list) > 1:
        job_description = items_list[-1].strip()
    items_list = re.split(a_str, job_description, 0)
    if len(items_list) > 1:
        basic_quals = items_list[0].strip()
    else:
        items_list = re.split('(Clearance|Build Your Career):', job_description, 0)
        basic_quals = items_list[0].strip()
    if basic_quals != '':
        quals_list = [un_msword_ify(q) for q in re.split('[\r\n]+', basic_quals, 0)]
        quals_list = [x for x in quals_list if x != '']
    quals_list = [x.strip() for x in quals_list if x.strip() != '']
    
    return quals_list

In [14]:

def print_emails(mask_series):
    for file_name in os.listdir(emails_dir):
        if file_name.endswith('.txt'):
            file_path = os.path.join(emails_dir, file_name)
            os.remove(file_path)
    for row_index, row_series in hunting_df[mask_series].sort_values('percent_fit', ascending=False).iterrows():
        req_str = row_series['Job Requisition']
        percent_fit = get_percent_fit(row_series)
        sents_list = ['I’m submitting my resume for {}.'.format(req_str),
                      'I’ve reviewed the basic qualifications and believe I’m a good fit for this project.',
                      'Below is a breakdown of the requirements and the amount of experience I have with each.',
                      'I’m available at your convenience to discuss my qualifications and look forward to hearing from you.']
        blurb_str = ' '.join(sents_list)
        hm_str = row_series['Hiring Manager']
        pr_str = row_series['Primary Recruiter']
        job_description = row_series['Job Description']
        quals_list = get_quals_list(job_description)
        quals_str = '\n•\t' + '\n•\t'.join(quals_list)
        file_path = os.path.join(emails_dir, '{}_email.txt'.format(row_series['Job Requisition ID'].strip()))
        if not os.path.isfile(file_path):
            with open(file_path, 'w', encoding=s.encoding_type) as io_wrapper:
                print('', file=io_wrapper)
                print('To: {}; {}'.format(clean_email(hm_str), clean_email(pr_str)), file=io_wrapper)
                print('CC: Safi, Claudia [USA] <safi_claudia@bah.com>; Borrelli, Bill [USA] <Borrelli_Bill@bah.com>', file=io_wrapper)
                print(subject_str.format(percent_fit, req_str), file=io_wrapper)
                print('', file=io_wrapper)
                print('Dear {},'.format(hm_str.split(' ')[0]), file=io_wrapper)
                print('', file=io_wrapper)
                print('{}'.format(blurb_str), file=io_wrapper)
                print('', file=io_wrapper)
                print('Basic Qualifications:{}'.format(quals_str), file=io_wrapper)
                print('', file=io_wrapper)
                print(concerns_str, file=io_wrapper)
                print('', file=io_wrapper)
                print('Attached: Dave_Babbitt_Resume_for_{}.pdf'.format('_'.join(re.split(r'[ \\\/:\*\?"><\|]+', req_str, 0))),
                      file=io_wrapper)
            !"{text_editor_path}" "{os.path.abspath(file_path)}"
    !start %windir%\explorer.exe "{os.path.abspath(emails_dir)}"

In [61]:

# Add new ORR to the hunting dataframe
text_editor_path = r'C:\Program Files\Notepad++\notepad++.exe'
jd_cn = 'Job Description'
reqid_cn = 'Job Requisition ID'
dupe_columns_list = ['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition', 'Job Requisition Type', 'Cluster', 'Time Type',
                     'Job Posting Title', 'Recruiting Start Date', 'Account Group', reqid_cn, 'Job Type',
                     'Supervisory Organization', 'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
                     'Resource Manager', 'Primary Location', jd_cn, 'Group', 'Job Profile', 'Job Family Group', 'FSO',
                     'Job Family', 'Job Requisition Status', 'Business Title', 'Job Posting', 'Primary Location Country',
                     'Required Clearance', 'Primary Recruiter']
import_columns_list = ['Job Posting', 'Job Requisition', reqid_cn, 'Job Requisition Status', 'Furthest Stage',
                       'Supervisory Organization', 'Group', 'Account Group', 'IMT', 'Cluster', 'FSO', 'Primary Recruiter',
                       'Resource Manager', 'Hiring Manager', 'Job Posting Title', 'Job Profile', 'Job Requisition Type',
                       'Management Level', 'Primary Location', 'Primary Location State/Province', 'Primary Location Country',
                       'Required Clearance', 'Clearance Agency', 'Time Type', 'Recruiting Start Date', 'Job Type', 'Job Family',
                       'Business Title', 'Job Family Group', jd_cn]
hunting_dir = r'D:\Documents\Administrivia\Job Hunting\csv'
def add_new_orr(file_name, hunting_df):
    file_path = os.path.join(hunting_dir, file_name)
    if os.path.isfile(file_path):
        df = pd.read_csv(file_path, header=0, skiprows=0, encoding='iso8859-1')
        df.columns = import_columns_list
        req_id_list = hunting_df[reqid_cn].unique().tolist()
        mask_series = (df[reqid_cn].isin(req_id_list))
        hunting_df = pd.concat([hunting_df, df[~mask_series]]).fillna({'is_opportunity_application_emailed': False})
        hunting_df[jd_cn] = hunting_df[jd_cn].map(un_msword_ify)
        hunting_df = hunting_df.drop_duplicates(subset=dupe_columns_list, ignore_index=True)
        hunting_df.reset_index(drop=True, inplace=True)
        
        return hunting_df

In [16]:

basic_quals_dict = s.load_object('basic_quals_dict')
def get_basic_quals(row_index):
    mask_series = (hunting_df.index == row_index)
    for row_index, row_series in hunting_df[mask_series].iterrows():
        percent_fit = get_percent_fit(row_series)
        job_description = row_series['Job Description']
        quals_list = get_quals_list(job_description)
        quals_str = get_quals_str(quals_list, basic_quals_dict)
        if (quals_str != '') and (percent_fit == 0):
            print('Basic Qualifications:{}'.format(quals_str))
            print_loc_computation(row_index, quals_list, basic_quals_dict)
    
    return quals_list


---
# Add Next ORR

In [59]:

file_name = 'Open Requisitions Report (ORR) 2020-03-27 09_02 EDT.csv'
print(file_name)
file_path = os.path.join(hunting_dir, file_name)
#!"{text_editor_path}" "{os.path.abspath(file_path)}"

Open Requisitions Report (ORR) 2020-03-27 09_02 EDT.csv


In [62]:

hunting_df = s.load_object('hunting_df')
hunting_df = add_new_orr(file_name, hunting_df)

In [63]:

hunting_df.tail(5).T

,3099,3100,3101,3102,3103
Hiring Manager,Sanjeevani Iyer (554498),Kate Knyzewski (520431),Kelly O'Fallon (545583),Kelly O'Fallon (545583),Kate Knyzewski (520431)
Management Level,Lead Associate,Senior Consultant,Senior Consultant,Associate,Associate
IMT,IS STRATEGY ANALYTICS &PS TECH,CYBER & ENGINEERING IMT,NA CONSULTING IMT,NA CONSULTING IMT,CYBER & ENGINEERING IMT
Job Requisition,R0067408 Workday Solutions Architect (Open),R0081029 Workforce Business Analyst (Open),"R0076955 Workforce Planner, Mid (Open)","R0076954 Workforce Planner, Senior (Open)","R0081215 Workforce Training Developer, Senior ..."
Job Requisition Type,ESG,Sold and Funded,Sold and Funded,Sold and Funded,Evergreen
Cluster,Wash Metro Cluster,Wash Metro Cluster,Wash Metro Cluster,Wash Metro Cluster,Wash Metro Cluster
Time Type,Full time,Full time,Full time,Full time,Full time
Job Posting Title,Workday Solutions Architect,Workforce Business Analyst,"Workforce Planner, Mid","Workforce Planner, Senior","Workforce Training Developer, Senior"
Safi Recommendation,NaN,NaN,NaN,NaN,NaN
Recruiting Start Date,2019-08-29,2020-03-20,2020-01-27,2020-01-27,2020-03-23


In [64]:

s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle



---
# Training

In [66]:

# Rebuild the datframe from the dictionary
rows_list = [{'qualification_str': qualification_str, 'is_fit': is_fit} for qualification_str, is_fit in basic_quals_dict.items()]
basic_quals_df = pd.DataFrame(rows_list)
s.store_objects(basic_quals_df=basic_quals_df)

# Re-transform the bag-of-words and tf-idf from the new manual scores
sents_list = basic_quals_df.qualification_str.tolist()

# Bag-of-words
cv = CountVectorizer(lowercase=True, tokenizer=regex_tokenizer, token_pattern=r'\b[1-9a-zA-Z][0-9a-zA-Z]*[#\+]{0,2}', ngram_range=(1, 3))
bow_matrix = cv.fit_transform(sents_list)
s.store_objects(bq_cv_vocab=cv.vocabulary_)

# Tf-idf, must get from BOW first
tt = TfidfTransformer()
tfidf_matrix = tt.fit_transform(bow_matrix)
s.store_objects(bq_tt=tt)

# Re-train the classifier
X = tfidf_matrix.toarray()
y = basic_quals_df.is_fit.to_numpy()
fit_estimators_dict = s.load_object('fit_estimators_dict')
#basic_quals_clf = RandomForestClassifier(n_estimators=997)
#basic_quals_clf = AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0, n_estimators=50, random_state=None)
basic_quals_clf = fit_estimators_dict['LogisticRegression']
basic_quals_clf.fit(X, y)
s.store_objects(basic_quals_clf=basic_quals_clf)

# Re-calibrate the inference engine
bq_cv_vocab = s.load_object('bq_cv_vocab')
bq_cv = CountVectorizer(vocabulary=bq_cv_vocab)
bq_cv._validate_vocabulary()
bq_tt = s.load_object('bq_tt')
def predict_percent_fit(quals_list):
    y_predict_proba = np.array([])
    if len(quals_list):
        X_test = bq_tt.transform(bq_cv.transform(quals_list)).toarray()
        y_predict_proba = basic_quals_clf.predict_proba(X_test)
    
    return y_predict_proba
print('Retraining complete')

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\basic_quals_df.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\bq_cv_vocab.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\bq_tt.pickle
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\basic_quals_clf.pickle
Retraining complete



---
# Inference

In [67]:

# Loop through all the unset %fit values, set them if you can, break for help if you can't
mask_series = (hunting_df.percent_fit >= 0.0)
for row_index, row_series in hunting_df[~mask_series].iterrows():
    quals_list, job_fitness = print_fit_job(row_index, row_series, basic_quals_dict)
    if job_fitness > 0.8:
        if all(qual_str in basic_quals_dict for qual_str in quals_list):
            hunting_df.loc[row_index, 'percent_fit'] = eval(' + '.join(map(qual_sum, quals_list))) / len(quals_list)
            s.store_objects(hunting_df=hunting_df)
        else:
            break
    else:
        if len(quals_list):
            hunting_df.loc[row_index, 'percent_fit'] = eval(' + '.join(map(qual_sum, quals_list))) / len(quals_list)
            s.store_objects(hunting_df=hunting_df)
print('{}/{} = {}% completed'.format(hunting_df[mask_series].shape[0], hunting_df.shape[0],
                                     int(100 * hunting_df[mask_series].shape[0] / hunting_df.shape[0])))

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle
Basic Qualifications:
quals_list[0] = "1+ years of experience with developing in C# using Visual Studio and .NET" (0.319948487016373)
quals_list[1] = "1+ years of experience with HTML, CSS, and JavaScript" (0.8416043522764212)
quals_list[2] = "Experience with developing in Microsoft Windows environments" (0.6370598790661617)
quals_list[3] = "Experience with SQL Server or Oracle" (0.8617958560698085)
quals_list[4] = "Ability to learn Pega" (0.6929794779538007)
quals_list[5] = "Active Top Secret clearance" (0.759169523493344)
quals_list[6] = "BA or BS degree" (0.8020448455208821)
0.8571428571428571

hunting_df.loc[1, 'percent_fit'] = (0+1+1+1+1+1+1)/7
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle
Basic Qualifications:
quals_list[0] = "5+ years of experience with developing in C# using Visual Studio and .NET" (0.319948487016373)
quals_list[1] = "5+ ye

In [181]:

# Manually label the unscored qual
qualification_str = quals_list[6]
print(qualification_str)
basic_quals_dict[qualification_str] = 1
s.store_objects(basic_quals_dict=basic_quals_dict)

Scheduled to obtain a BA or BS degree in Spring or Fall 2020, Spring or Fall 2021 or Spring or Fall 2022
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\basic_quals_dict.pickle



---
# Create the emails

In [19]:

def f(x):
    
    return 'python' in str(x).lower()

def g(x):
    
    return 'polygraph' in str(x).lower()

ds_regex = re.compile(r'data *scien')
ml_regex = re.compile(r'machine *learning|\bML\b')
def ff(x):
    x = str(x).lower()
    if ds_regex.search(x) or ml_regex.search(x) or ('python' in x):
        match = True
    else:
        match = False
    
    return match

mask_series = hunting_df['Job Requisition Type'].isin(['Sold and Funded', 'Sold & Unfunded'])
#mask_series = mask_series & hunting_df['Job Description'].map(f)
mask_series = mask_series & hunting_df['Job Description'].map(ff)
mask_series = mask_series & (hunting_df.percent_fit >= 0.85) & ~hunting_df['is_opportunity_application_emailed']
mask_series = mask_series & ~(hunting_df.is_remote_delivery == False)
mask_series = mask_series & ~hunting_df['Required Clearance'].isin(['TS/SCI', 'TS/SCI w/CIP', 'TS/SCI w/FSP'])
mask_series = mask_series & ~hunting_df['Job Description'].map(g)
print(hunting_df[mask_series].shape)
print(hunting_df[mask_series].groupby('Required Clearance').count().T.max().sort_values(ascending=False))

(0, 36)
Series([], dtype: float64)


In [ ]:

hunting_df[mask_series].head(5).T

In [20]:

print_emails(mask_series)

In [194]:

# Manually note the email has been sent
req_id = 'R0081074'.strip()
mask_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle



---
# Flag setting

In [34]:

# Set notes for this one job
req_id = 'R0063024'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
manager_notes = hunting_df.loc[mask_series, 'manager_notes'].tolist()[0]
if len(manager_notes):
    manager_notes += ' '
manager_notes += 'Due to the hiring freeze that job req was recently taken down.'
manager_notes += ''
hunting_df.loc[mask_series, 'manager_notes'] = manager_notes
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [35]:

# Set remote delivery for this one job
req_id = 'R0063024'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [2]:

hunting_df.columns

Index(['Hiring Manager', 'Management Level', 'IMT', 'Job Requisition',
       'Job Requisition Type', 'Cluster', 'Time Type', 'Job Posting Title',
       'Safi Recommendation', 'Recruiting Start Date', 'Account Group',
       'Job Requisition ID', 'Job Type', 'Supervisory Organization',
       'Clearance Agency', 'Primary Location State/Province', 'Furthest Stage',
       'Resource Manager', 'Primary Location', 'Job Description', 'Group',
       'Job Profile', 'Job Family Group', 'FSO', 'Job Family',
       'Job Requisition Status', 'Business Title', 'Job Posting',
       'Primary Location Country', 'Required Clearance', 'Primary Recruiter',
       'percent_fit', 'is_opportunity_application_emailed', 'opportunity_application_email_date',
       'is_remote_delivery', 'manager_notes', 'CS Notes'],
      dtype='object')

In [25]:

column_name = 'Primary Recruiter'
last_name = 'Sallis'.lower()
first_name = 'Michael'.lower()

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

column_name = 'Hiring Manager'
last_name = 'Arunachalam'.lower()
first_name = 'Siva'.lower()

def g(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

mask_series = hunting_df[column_name].map(f) & hunting_df[column_name].map(g)
req_id_list = hunting_df[mask_series]['Job Requisition ID'].tolist()
formatted_str = 'Thanks for the feedback. I have marked all the reqs you are associated with ({}).'
print(formatted_str.format(conjunctify_list(req_id_list)))
mask_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Thanks for the feedback. I have marked all the reqs you are associated with (R0073719, R0073918, R0074423, R0073696, R0073709, R0073944, R0073943, R0073942, R0073940, R0073939, R0073718, R0073808, R0074424, R0073711, R0076554, R0079010, R0079006, and R0081219).
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [208]:

# Currently all our roles are local to the DC Metro Area.
req_id = 'R0079210'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
location = hunting_df[mask_series]['Primary Location'].tolist()[0]
manager = hunting_df[mask_series]['Hiring Manager'].tolist()[0]
mask_series = (hunting_df['Primary Location'] == location)
print('Is she talking about the {} reqs in her location?'.format(hunting_df[mask_series].shape[0]))
mask_series = (hunting_df['Hiring Manager'] == manager)
print('Is she talking about the {} reqs she is managing?'.format(hunting_df[mask_series].shape[0]))
mask_series = (hunting_df['Primary Location'] == location) & (hunting_df['Hiring Manager'] == manager)
formatted_str = 'Thank you for the feedback. I am assuming you are talking about the {} reqs you are managing at your location ({}).'

# Set remote delivery for this list of jobs
req_id_list = hunting_df[mask_series]['Job Requisition ID'].tolist()
mask_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)
print(formatted_str.format(hunting_df[mask_series].shape[0], conjunctify_list(req_id_list)))

Is she talking about the 121 reqs in her location?
Is she talking about the 2 reqs she is managing?
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle
Thank you for the feedback. I am assuming you are talking about the 2 reqs you are managing at your location (R0071717 and R0079210).


In [53]:

column_name = 'Hiring Manager'
last_name = 'Cofrancesco'.lower()
first_name = 'Elaine'.lower()

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

mask_series = hunting_df[column_name].map(f)
req_id_list = hunting_df[mask_series]['Job Requisition ID'].tolist()
formatted_str = 'Thanks for the feedback. Does this include all the reqs you are associated with ({})?'
print(formatted_str.format(conjunctify_list(req_id_list)))
mask_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Thanks for the feedback. Does this include all the reqs you are associated with (R0080347)?
Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [289]:

column_name = 'Hiring Manager'
last_name = 'Jendzejec'.lower()
first_name = 'Alexander'.lower()

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

mask_series = hunting_df[column_name].map(f)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [89]:

column_name = 'Hiring Manager'
last_name = 'ferraro'
first_name = 'mike'

def f(x):
    x = str(x).lower()
    
    return (first_name in x) and (last_name in x)

mask_series = hunting_df[column_name].map(f)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [81]:

# Set remote delivery for this list of jobs
req_id_list = ['R0072627', 'R0072630', 'R0072685', 'R0072688', 'R0072699', 'R0072700', 'R0078787', 'R0078792', 'R0078793', 'R0072670',
               'R0072672', 'R0078798', 'R0078799']
mask_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [38]:

# Set remote delivery for this list of jobs
req_id_list = ['R0073564', 'R0073583', 'R0073584', 'R0073585', 'R0073586']
mask_series = hunting_df['Job Requisition ID'].isin(req_id_list)
hunting_df.loc[mask_series, 'is_remote_delivery'] = True
hunting_df.loc[mask_series, 'is_opportunity_application_emailed'] = True
s.store_objects(hunting_df=hunting_df)

Pickling to D:\Documents\Repositories\notebooks\Miscellaneous\saves\pickle\hunting_df.pickle


In [ ]:

# Set university recruiting for this job
req_id = 'R0066388'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[mask_series, 'is_for_university_recruiting'] = 1
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Manually note you can't do this job from home
req_id = 'R0064764'
mask_series = (hunting_df['Job Requisition ID'] == req_id)
hunting_df.loc[mask_series, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

In [ ]:

# Manually note you can't do this job from home
hunting_df.loc[83, 'is_remote_delivery'] = False
s.store_objects(hunting_df=hunting_df)

In [ ]:

mask_series = (hunting_df.index == 2616)
hunting_df.loc[mask_series, 'is_for_university_recruiting'] = 1
hunting_df.loc[mask_series, 'percent_fit'] = 0.0
s.store_objects(hunting_df=hunting_df)